<a href="https://colab.research.google.com/github/michal-dom/praca_magisterska/blob/master/disaster_tweets/DTC-EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from os import listdir
from os.path import isfile, join
files = [f for f in listdir('/content/drive/My Drive/Studia/magisterka/disaster tweets')]
print(files)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
['test.csv', 'train.csv']


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
plt.style.use('ggplot')

In [3]:
from nltk import download
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

download('stopwords')
download('punkt')
stop=set(stopwords.words('english'))
import re

import gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from collections import defaultdict
from collections import  Counter

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
from tqdm import tqdm

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from keras.optimizers import Adam

Using TensorFlow backend.


In [24]:
tweet= pd.read_csv('/content/drive/My Drive/Studia/magisterka/disaster tweets/train.csv')
test=pd.read_csv('/content/drive/My Drive/Studia/magisterka/disaster tweets/test.csv')
tweet.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
tweet.memory_usage() 

Index         128
id          60904
keyword     60904
location    60904
text        60904
target      60904
dtype: int64

In [13]:
tweet.shape

(7613, 5)

In [15]:
test.shape

(3263, 4)

In [17]:
len(stop)

179

In [0]:
words_dict = {}

for text in tweet['text']:
  for w in text.split(' '):
    if not w in words_dict:
      words_dict[w] = 1
    else:
      words_dict[w] += 1


In [23]:
print(words_dict)

{'Our': 24, 'Deeds': 1, 'are': 345, 'the': 2573, 'Reason': 5, 'of': 1721, 'this': 335, '#earthquake': 8, 'May': 13, 'ALLAH': 1, 'Forgive': 1, 'us': 52, 'all': 178, 'Forest': 28, 'fire': 147, 'near': 50, 'La': 11, 'Ronge': 1, 'Sask.': 1, 'Canada': 4, 'All': 50, 'residents': 4, 'asked': 9, 'to': 1804, "'shelter": 1, 'in': 1757, "place'": 1, 'being': 67, 'notified': 1, 'by': 467, 'officers.': 1, 'No': 50, 'other': 31, 'evacuation': 32, 'or': 181, 'shelter': 4, 'place': 22, 'orders': 11, 'expected': 11, '13,000': 1, 'people': 146, 'receive': 2, '#wildfires': 5, 'California': 89, '': 770, 'Just': 46, 'got': 103, 'sent': 12, 'photo': 8, 'from': 371, 'Ruby': 1, '#Alaska': 2, 'as': 282, 'smoke': 33, 'pours': 1, 'into': 162, 'a': 1840, 'school': 46, '#RockyFire': 4, 'Update': 5, '=>': 1, 'Hwy.': 4, '20': 13, 'closed': 15, 'both': 20, 'directions': 1, 'due': 27, 'Lake': 8, 'County': 30, '-': 762, '#CAfire': 1, '#flood': 3, '#disaster': 5, 'Heavy': 7, 'rain': 27, 'causes': 10, 'flash': 14, 'flood

In [0]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


tweet['text']=tweet['text'].apply(lambda x : remove_URL(x))

In [0]:
def remove_ATs(text):
    url = re.compile(r'\B@\w+')
    return url.sub(r'',text)

tweet['text']=tweet['text'].apply(lambda x : remove_ATs(x))

In [0]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

tweet['text']=tweet['text'].apply(lambda x : remove_html(x))

In [0]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

tweet['text']=tweet['text'].apply(lambda x: remove_emoji(x))

In [0]:
def remove_punct(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


tweet['text']=tweet['text'].apply(lambda x : remove_punct(x))

In [0]:
def remove_new_lines(text):
    return re.sub(r'[\n]', ' ', text)


tweet['text']=tweet['text'].apply(lambda x : remove_new_lines(x))

In [0]:
def remove_spaces(text):
    html=re.compile(r' +')
    return html.sub(r' ',text)

tweet['text']=tweet['text'].apply(lambda x : remove_spaces(x))

In [0]:
def to_lower(text):
    return str(text).lower()

tweet['text']=tweet['text'].apply(lambda x : to_lower(x))

In [33]:
for text in tweet['text']:
  print(text)
  print('\n\n\n')

Streaming output truncated to the last 5000 lines.




In islam saving a person is equal in reward to saving all humans Islam is the opposite of terrorism




OMEGA MEN Writer Explores Terrorism Religion In an Epic KYLE RAYNER Story  Newsarama




In islam saving a person is equal in reward to saving all humans Islam is the opposite of terrorism




 Arrest report Affiliation with Terrorism




Truth  News BBC CNN Islam Truth god ISIS terrorism Quran Lies 




Really cannot condemn an entire group based on the actions of a few A heartwarming unity against terrorism 




Truth  News BBC CNN Islam Truth god ISIS terrorism Quran Lies 




  Hes already done it by negotiating with the 1 state of terrorism in the World What was his hurry in trying to get a deal




 Articles In Saudi Press Reject Russian Initiative For Regional Alliance With Assad Regime To Fight Terrorism




Truth  News BBC CNN Islam Truth god ISIS terrorism Quran Lies 




Pakistan Supreme Court endorses military courts 

In [36]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus


corpus=create_corpus(tweet)

100%|██████████| 7613/7613 [00:01<00:00, 7233.00it/s]


In [39]:
print(corpus[17])

['summer', 'lovely']


In [0]:
words_dict = {}

for text in tweet['text']:
  for w in text.split(' '):
    if not w in words_dict:
      words_dict[w] = 1
    else:
      words_dict[w] += 1


In [41]:
print(words_dict)

{'our': 100, 'deeds': 2, 'are': 402, 'the': 3269, 'reason': 20, 'of': 1828, 'this': 478, 'earthquake': 50, 'may': 88, 'allah': 9, 'forgive': 2, 'us': 164, 'all': 258, 'forest': 65, 'fire': 252, 'near': 54, 'la': 26, 'ronge': 1, 'sask': 1, 'canada': 11, 'residents': 8, 'asked': 9, 'to': 1946, 'shelter': 6, 'in': 1979, 'place': 26, 'being': 90, 'notified': 1, 'by': 525, 'officers': 8, 'no': 253, 'other': 38, 'evacuation': 50, 'or': 203, 'orders': 11, 'expected': 15, '13000': 4, 'people': 195, 'receive': 2, 'wildfires': 11, 'california': 117, '': 5323, 'just': 320, 'got': 112, 'sent': 13, 'photo': 41, 'from': 421, 'ruby': 1, 'alaska': 5, 'as': 330, 'smoke': 48, 'pours': 1, 'into': 173, 'a': 2183, 'school': 66, 'rockyfire': 4, 'update': 37, 'hwy': 9, '20': 26, 'closed': 20, 'both': 27, 'directions': 1, 'due': 31, 'lake': 14, 'county': 38, 'cafire': 2, 'flood': 56, 'disaster': 153, 'heavy': 20, 'rain': 44, 'causes': 13, 'flash': 21, 'flooding': 50, 'streets': 8, 'manitou': 1, 'colorado': 16

In [18]:
max(words_dict, key=words_dict.get)


''